In [21]:
from pymongo import MongoClient
import pprint

import pandas as pd

# Requests sends and recieves HTTP requests.
import requests

# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup

import json
import time

In [22]:
# roster_urls = {}
# roster_urls['unc'] = 'https://goheels.com/sports/womens-basketball/roster/'
# roster_urls['duke'] = 'https://goduke.com/sports/womens-basketball/roster/'
# roster_urls['miami'] = 'https://hurricanesports.com/sports/womens-basketball/roster/'
# roster_urls['nc_state'] = 'https://gopack.com/sports/womens-basketball/roster/'

# #List of desired seasons of data - will combine with roster_urls to scrape
# seasons = ['2009-10', '2010-11', '2011-12', '2017-18', '2018-19', '2019-20']

In [23]:
#Grab requests and input into MongoDB - commented out after completing once
client = MongoClient('localhost', 27017)
db = client.wbb2
rosters = db.rosters

# for season in seasons:
#     for team, url in roster_urls.items():
#         r = requests.get(url + season)
#         rosters.insert_one({'team': team,
#                             'season': season,
#                             'html': r.text})
#         time.sleep(2)

At this point I have a cursor that contains all the HTML files containing each team's roster for every season.

Now I need to separate out the html but make sure to tie it to the correct team & season.

Fields I want are:
-heights
-names
-positions
-class_yr

Update MongoDB fields to have the keys with the fields you want with values that consist of the parsed out values.

*****Note*****
-Ran into an issue with parsing the data from Mongo.  I saved the html files in mongo using r.content.  R.content saves the files as bytes rather than strings (r.text).  Didn't find a quick solution to deal with bytes so pulled the html again into the 'wbb2' database and saved it using r.text.

--Update--
The data is fine for unc for the 2019-2020, 2018-19, 2017-2018 seasons, but I suspect the HTML changed so now I have to figure that out.  This wasn't apparent the first time because I was checking the oldest seasons as opposed to the last seasons.

-Iterate from the end of the cursor and once the lists fail to populate, use a different scheme for parsing.

In [ ]:
cursor = rosters.find({})

In [25]:
seasons_2 = ['2017-18', '2018-19', '2019-20']
all_teams = []
for item in cursor:
    if item['season'] in seasons_2 or item['team'] == 'duke':
        soup = BeautifulSoup(item['html'], 'html.parser')
        #run parsing code
        heights = [item.text for item in soup.find_all(class_='height')]
        names = [item.text for item in soup.find_all(class_='sidearm-table-player-name')]
        positions = [item.text for item in soup.find_all(class_='rp_position_short')]
        class_yr = [item.text for item in soup.find_all(class_='roster_class')]
        all_teams.append({'team': item['team'],
                          'season': item['season'],
                          'heights': heights,
                          'names': names,
                          'positions': positions,
                          'class_yr': class_yr})
    else:
        soup = BeautifulSoup(cursor[num]['html'], 'html.parser')
        #run parsing code
        heights = [item.text for item in soup.find_all(class_='sidearm-roster-player-height')]
        first_names = [item.text for item in soup.find_all(class_='sidearm-roster-player-first-name')]
        last_names = [item.text for item in soup.find_all(class_='sidearm-roster-player-last-name')]
        positions = [item.text for item in soup.find_all(class_='sidearm-list-card-details-item sidearm-roster-player-position-short')]
        class_yr = [item.text for item in soup.find_all(class_='sidearm-roster-player-academic-year')]
        all_teams.append({'team': item['team'],
                          'season': item['season'],
                          'heights': heights,
                          'names': names,
                          'positions': positions,
                          'class_yr': class_yr})

#******************OLD CODE - Adjust for HTML changes***************
# all_teams = []
# for item in cursor:
#     if item['team'] == 'unc':
#         soup = BeautifulSoup(item['html'], 'html.parser')
#         #run unc code
#         heights = [item.text for item in soup.find_all(class_='height')]
#         names = [item.text for item in soup.find_all(class_='sidearm-table-player-name')]
#         positions = [item.text for item in soup.find_all(class_='rp_position_short')]
#         class_yr = [item.text for item in soup.find_all(class_='roster_class')]
#         all_teams.append({'team': item['team'],
#                           'season': item['season'],
#                           'heights': heights,
#                           'names': names,
#                           'positions': positions,
#                           'class_yr': class_yr})
#     elif item['team'] == 'duke':
#         soup = BeautifulSoup(item['html'], 'html.parser')
#         #run duke code
#         heights = [item.text for item in soup.find_all(class_='height')]
#         names = [item.text for item in soup.find_all(class_='sidearm-table-player-name')]
#         positions = [item.text for item in soup.find_all(class_='rp_position_short')]
#         class_yr = [item.text for item in soup.find_all(class_='roster_class')]
#         all_teams.append({'team': item['team'],
#                           'season': item['season'],
#                           'heights': heights,
#                           'names': names,
#                           'positions': positions,
#                           'class_yr': class_yr})
#     elif item['team'] == 'miami':
#         soup = BeautifulSoup(item['html'], 'html.parser')
#         #run miami code
#         heights = [item.text for item in soup.find_all(class_='height')]
#         names = [item.text for item in soup.find_all(class_='sidearm-table-player-name')]
#         positions = [item.text for item in soup.find_all(class_='rp_position_short')]
#         class_yr = [item.text for item in soup.find_all(class_='roster_class')]
#         all_teams.append({'team': item['team'],
#                           'season': item['season'],
#                           'heights': heights,
#                           'names': names,
#                           'positions': positions,
#                           'class_yr': class_yr})
#     else:
#         soup = BeautifulSoup(item['html'], 'html.parser')
#         #run nc_state code
#         heights = [item.text for item in soup.find_all(class_='height')]
#         names = [item.text for item in soup.find_all(class_='sidearm-table-player-name')]
#         positions = [item.text for item in soup.find_all(class_='rp_position_short')]
#         class_yr = [item.text for item in soup.find_all(class_='roster_class')]
#         all_teams.append({'team': item['team'],
#                           'season': item['season'],
#                           'heights': heights,
#                           'names': names,
#                           'positions': positions,
#                           'class_yr': class_yr})

Need to verify all records were correctly pulled by checking for blanks and storing the teams & indices.

In [30]:
blank_teams = []
blank_indices = []
i=0
for item in all_teams:
    if item['heights'] == []:
        blank_teams.append({'team': item['team'],
                            'season': item['season']
        })
        blank_indices.append(i)
        i+=1
    else:
        i+=1

print(blank_indices)
blank_teams

[0, 2, 3, 4, 6, 7, 8, 10, 11]


[{'team': 'unc', 'season': '2009-10'},
 {'team': 'miami', 'season': '2009-10'},
 {'team': 'nc_state', 'season': '2009-10'},
 {'team': 'unc', 'season': '2010-11'},
 {'team': 'miami', 'season': '2010-11'},
 {'team': 'nc_state', 'season': '2010-11'},
 {'team': 'unc', 'season': '2011-12'},
 {'team': 'miami', 'season': '2011-12'},
 {'team': 'nc_state', 'season': '2011-12'}]

In [35]:
soup = BeautifulSoup(cursor[0]['html'], 'html.parser')
print(soup.prettify())

<!DOCTYPE doctype html>
<html class="wide roster-page wbball sport wbball" id="ctl00_html" lang="en">
 <head>
  <script>
   window.client_hostname = "goheels.com";window.server_name = "866410-LB15";window.client_division = "DI";window.targetting_criteria = {"page_template":"roster-list","sport_name":"wbball","sport_name_custom":"wbball","site":"unc","cbs_site_code":"unc","content_id":null,"sect":"wbaskbl"};window.client_shortname = "unc";window.dfp_network_code = "29658103";window.img_dfp_unit_name = "";window.block_dfp = false;window.reactions = null;window.cbs_ads_activated = true;window.site_colors = {"safe_text_white":"#13294b","safe_text_black":"#ffffff","primary_background":"#13294b","primary_text":"#7bafd4","secondary_background":"#7bafd4","secondary_text":"#13294b"};
                    window.utagData = function() {
                        return {
                            embedPageUrl: location.href,
                            brandPlatformId: "sidearm_site_" + (document.

In [37]:
# for num in blank_indices:
#         soup = BeautifulSoup(cursor[num]['html'], 'html.parser')
#         #run parsing code
#         heights = [item.text for item in soup.find_all(class_='sidearm-roster-player-height')]
#         first_names = [item.text for item in soup.find_all(class_='sidearm-roster-player-first-name')]
#         last_names = [item.text for item in soup.find_all(class_='sidearm-roster-player-last-name')]
#         positions = [item.text for item in soup.find_all(class_='sidearm-list-card-details-item sidearm-roster-player-position-short')]
#         class_yr = [item.text for item in soup.find_all(class_='sidearm-roster-player-academic-year')]


soup = BeautifulSoup(cursor[num]['html'], 'html.parser')
#run parsing code
heights = [item.text for item in soup.find_all(class_='sidearm-roster-player-height')]
first_names = [item.text for item in soup.find_all(class_='sidearm-roster-player-first-name')]
last_names = [item.text for item in soup.find_all(class_='sidearm-roster-player-last-name')]
positions = [item.text for item in soup.find_all(class_='sidearm-list-card-details-item sidearm-roster-player-position-short')]
class_yr = [item.text for item in soup.find_all(class_='sidearm-roster-player-academic-year')]

#         all_teams.append({'team': item['team'],
#                           'season': item['season'],
#                           'heights': heights,
#                           'names': names,
#                           'positions': positions,
#                           'class_yr': class_yr})
print(heights)
print(first_names)
print(last_names)
print(positions)
print(class_yr)
len(heights)

['5\'4"', '5\'5"', '5\'5"', '6\'3"', '5\'11"', '5\'8"', '5\'9"', '5\'2"', '5\'11"', '5\'11"', '5\'9"', '6\'4"', '6\'2"', '6\'2"', '6\'1"', '5\'4"', '5\'5"', '5\'5"', '6\'3"', '5\'11"', '5\'8"', '5\'9"', '5\'2"', '5\'11"', '5\'11"', '5\'9"', '6\'4"', '6\'2"', '6\'2"', '6\'1"']
['Myisha', "Len'Nique", 'Kim', 'Tia', 'Breezy', 'Emili', 'Krystal', 'Paige', 'Erica', 'Bonae', 'Marissa', 'Lakeesa', 'Therany', 'Kody', 'Kiana']
['Goodwin-Coleman', 'Brown', 'Durham', 'Bell', 'Williams', 'Tasler', 'Barrett', 'Woodard', 'Donovan', 'Holston', 'Kastanek', 'Daniel', 'Dunnigan', 'Burke', 'Evans']
['\r\n                                                                Guard\r\n                                                            ', '\r\n                                                                Guard\r\n                                                            ', '\r\n                                                                Guard\r\n                                                    

30